In [24]:
import pandas as pd
import sqlite3
import matplotlib.pyplot as plt
import numpy as np
import re
import seaborn as sns
import time
import requests
from sklearn.preprocessing import LabelEncoder
from scipy import stats
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_columns', None)

This notebook will clean up the combined public housing buildings/developments/agencies datafile so that it is ready to be merged with the FEMA NFIRS dataset in the subsequent dataset.

In [25]:
public = pd.read_csv('../data/buildings_dev_pha_2.1.csv')

C:\Users\jackv\AppData\Local\Temp\ipykernel_10716\407126396.py:1: DtypeWarning: Columns (104) have mixed types. Specify dtype option on import or set low_memory=False.
  public = pd.read_csv('./buildings_dev_pha_2.0.csv')


In [26]:
public.sample()

,x,y,objectid,participant_code,formal_participant_name,development_code,project_name,building_name,building_number,building_type_code,building_status_type_code,national_bldg_id,construct_date,dofa_actual_dt,total_dwelling_units,acc_units,total_occupied,regular_vacant,total_units,pct_occupied,number_reported,pct_reported,months_since_report,pct_movein,people_per_unit,people_total,rent_per_month,spending_per_month,spending_per_month_prev_yr,hh_income,person_income,pct_lt5k,pct_5k_lt10k,pct_10k_lt15k,pct_15k_lt20k,pct_ge20k,pct_wage_major,pct_welfare_major,pct_other_major,pct_median,pct_lt50_median,pct_lt30_median,pct_2adults,pct_1adult,pct_female_head,pct_female_head_child,pct_disabled_lt62,pct_disabled_ge62,pct_disabled_all,pct_lt24_head,pct_age25_50,pct_age51_61,pct_age62plus,pct_age85plus,pct_minority,pct_black,pct_native_american,pct_asian,pct_hispanic,months_waiting,months_from_movein,pct_utility_allow,ave_util_allow,pct_bed1,pct_bed2,pct_bed3,pct_overhoused,tminority,tpoverty,tpct_ownsfd,chldrn_mbr_cnt,eldly_prcnt,pct_disabled_lt62_all,pct_lt80_median,median_inc_amnt,state2kx,cnty_nm2kx,cnty2kx,tract2kx,bg2kx,block2kx,dpvact,dpvnost,curcnty_nm,curcnty,curcosub,curcosub_nm,place2kx,place_nm2kx,place_cc2kx,place_inc2kx,msa,msa_nm,cbsa,cbsa_nm,necta,necta_nm,metro,micro,fcd_fips91,hlc,dpv,dpvrc,std_addr,urb_out,std_city,std_st,std_zip5,std_zip9,zip_class,zcta2kx,dpbc,dpbc_cksum,std_zip11,addr_type,apt_type,c1pgrc,c1pprb,c1pdrc,c1psrc,c1parc,c1pzrc,msgusps,lat,lon,rc2kx,stm2kx,lvl2kx,ur,msg2kx,county_level,place_level,tract_level,blkgrp_level,rad_chap_indr,rad_type,last_updt_dttm,annl_expns_amnt,annl_expns_amnt_prev_yr,pha_total_units,dev_buil_nu_entrance,ha_phn_num,ha_fax_num,ha_email_addr_text,exec_dir_phone,exec_dir_fax,exec_dir_email,dev_development_code,dev_spending_per_month_prev_yr,dev_chldrn_mbr_cnt,dev_annl_expns_amnt,dev_annl_expns_amnt_prev_yr,dev_eldly_prcnt,dev_pct_disabled_lt62_all,dev_pct_lt80_median,dev_median_inc_amnt,dev_scattered_site_ind,dev_pd_status_type_code,dev_total_units,dev_total_dwelling_units,dev_acc_units,dev_total_occupied,dev_regular_vacant,dev_pha_total_units,dev_pct_occupied,dev_number_reported,dev_pct_reported,dev_months_since_report,dev_pct_movein,dev_people_per_unit,dev_people_total,dev_rent_per_month,dev_spending_per_month,dev_hh_income,dev_person_income,dev_pct_lt5k,dev_pct_5k_lt10k,dev_pct_10k_lt15k,dev_pct_15k_lt20k,dev_pct_ge20k,dev_pct_wage_major,dev_pct_welfare_major,dev_pct_other_major,dev_pct_median,dev_pct_lt50_median,dev_pct_lt30_median,dev_pct_2adults,dev_pct_1adult,dev_pct_female_head,dev_pct_female_head_child,dev_pct_disabled_lt62,dev_pct_disabled_ge62,dev_pct_disabled_all,dev_pct_lt24_head,dev_pct_age25_50,dev_pct_age51_61,dev_pct_age62plus,dev_pct_age85plus,dev_pct_minority,dev_pct_black,dev_pct_native_american,dev_pct_asian,dev_pct_hispanic,dev_months_waiting,dev_months_from_movein,dev_pct_utility_allow,dev_ave_util_allow,dev_pct_bed1,dev_pct_bed2,dev_pct_bed3,dev_pct_overhoused,dev_tminority,dev_tpoverty,dev_tpct_ownsfd,pha_participant_code,pha_phas_designation,pha_ha_low_rent_size_category,pha_section8_units_cnt,pha_ha_section_8_size_category,pha_ha_combined_size_category,pha_ha_fye,pha_ha_program_type,pha_section8_occupied,pha_opfund_amnt,pha_opfund_amnt_prev_yr,pha_capfund_amnt,pha_capfund_amnt_prev_yr,pha_ross_amnt,pha_fss_amnt,pha_chldrn_mbr_cnt,pha_eldly_prcnt,pha_pct_disabled_lt62_all,pha_pct_lt80_median,pha_median_inc_amnt,pha_annl_expns_amnt,pha_annl_expns_amnt_prev_yr,corrected_address,corrected_unit
126037,-111.010191,32.138674,126038,AZ004,Housing and Community Development Tucson,AZ004000051,"POSADAS SENTINEL, PH. I",LOGGER,590,SF,INAPCP,5.944200e+09,2000/10/20 00:00:00+00,2002/09/30 00:00:00+00,1,1,1,0,1,100.0,1,-4.0,-4,-4.0,-4.0,-4,-4,-4,-4,-4,-4,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4,-4,-4.0,-4,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4.0,-4,-4.0,-4.0,-4.0,-4,4.0,Pi

-4 indicates the value was anonymized because the property had 10 or fewer residents.

In [27]:
public = public.replace(-4, np.nan)

In [28]:
pd.set_option('display.max_rows', 300)
display((public.isna().sum() / len(public)).sort_values(ascending=False))
pd.set_option('display.max_rows', 10)

dpvrc                             1.000000
dpv                               1.000000
place_inc2kx                      1.000000
dpvnost                           1.000000
dpvact                            1.000000
urb_out                           0.999406
zip_class                         0.993871
necta_nm                          0.967178
annl_expns_amnt_prev_yr           0.951648
pct_disabled_lt62                 0.951528
pct_age62plus                     0.951528
pct_age51_61                      0.951528
pct_age25_50                      0.951528
pct_lt24_head                     0.951528
pct_disabled_all                  0.951528
pct_disabled_ge62                 0.951528
pct_15k_lt20k                     0.951528
pct_minority                      0.951528
pct_female_head_child             0.951528
pct_female_head                   0.951528
pct_1adult                        0.951528
pct_2adults                       0.951528
pct_lt30_median                   0.951528
pct_age85pl

In [29]:
pd.set_option('display.max_rows', 300)
display(public.dtypes)
pd.set_option('display.max_rows', 10)

x                                 float64
y                                 float64
objectid                            int64
participant_code                   object
formal_participant_name            object
development_code                   object
project_name                       object
building_name                      object
building_number                    object
building_type_code                 object
building_status_type_code          object
national_bldg_id                  float64
construct_date                     object
dofa_actual_dt                     object
total_dwelling_units                int64
acc_units                           int64
total_occupied                      int64
regular_vacant                      int64
total_units                         int64
pct_occupied                      float64
number_reported                     int64
pct_reported                      float64
months_since_report               float64
pct_movein                        

In [30]:
columns_to_drop = [
    'x', 
    'y', 
    'formal_participant_name', 
    'project_name',
    'building_name',
    'building_number',
    'national_bldg_id',
    'state2kx', 
    'cnty_nm2kx',
    'cnty2kx',
    'tract2kx',
    'bg2kx',
    'block2kx',
    'dpvact',
    'dpvnost', 
    'curcnty_nm', 
    'curcnty', 
    'curcosub', 
    'curcosub_nm',
    'place2kx', 
    'place_nm2kx', 
    'place_cc2kx', 
    'place_inc2kx', 
    'msa', 
    'cbsa', 
    'cbsa_nm', 
    'necta', 
    'necta_nm', 
    'zcta2kx',
    'dpbc',
    'dpbc_cksum',
    'c1pgrc',  
    'c1pdrc',
    'c1psrc',
    'c1parc',
    'c1pzrc',
    'lat',
    'lon',
    'rc2kx',
    'stm2kx',
    'fcd_fips91',
    'hlc',
    'dpv', 
    'dpvrc', 
    'std_addr',
    'urb_out', 
    'msg2kx',
    'county_level',
    'place_level',	
    'tract_level',	
    'blkgrp_level',
    'county_level',
    'place_level',
    'tract_level',
    'blkgrp_level',
    'dev_buil_nu_entrance',
    'ha_phn_num',
    'ha_fax_num',
    'ha_email_addr_text',
    'exec_dir_phone' ,
    'exec_dir_fax',
    'exec_dir_email',
    'dev_development_code',
    'pha_participant_code'
]

public = public.drop(columns=columns_to_drop)

In [31]:
public['construction_year'] = pd.to_datetime(public['construct_date']).dt.year
public['dofa_actual_year'] = pd.to_datetime(public['dofa_actual_dt']).dt.year
public = public.drop(['construct_date', 'dofa_actual_dt'], axis=1)

In [32]:
public.building_type_code = public.building_type_code.str.upper()

In [33]:
public.sample()

,objectid,participant_code,development_code,building_type_code,building_status_type_code,total_dwelling_units,acc_units,total_occupied,regular_vacant,total_units,pct_occupied,number_reported,pct_reported,months_since_report,pct_movein,people_per_unit,people_total,rent_per_month,spending_per_month,spending_per_month_prev_yr,hh_income,person_income,pct_lt5k,pct_5k_lt10k,pct_10k_lt15k,pct_15k_lt20k,pct_ge20k,pct_wage_major,pct_welfare_major,pct_other_major,pct_median,pct_lt50_median,pct_lt30_median,pct_2adults,pct_1adult,pct_female_head,pct_female_head_child,pct_disabled_lt62,pct_disabled_ge62,pct_disabled_all,pct_lt24_head,pct_age25_50,pct_age51_61,pct_age62plus,pct_age85plus,pct_minority,pct_black,pct_native_american,pct_asian,pct_hispanic,months_waiting,months_from_movein,pct_utility_allow,ave_util_allow,pct_bed1,pct_bed2,pct_bed3,pct_overhoused,tminority,tpoverty,tpct_ownsfd,chldrn_mbr_cnt,eldly_prcnt,pct_disabled_lt62_all,pct_lt80_median,median_inc_amnt,msa_nm,metro,micro,std_city,std_st,std_zip5,std_zip9,zip_class,std_zip11,addr_type,apt_type,c1pprb,msgusps,lvl2kx,ur,rad_chap_indr,rad_type,last_updt_dttm,annl_expns_amnt,annl_expns_amnt_prev_yr,pha_total_units,dev_spending_per_month_prev_yr,dev_chldrn_mbr_cnt,dev_annl_expns_amnt,dev_annl_expns_amnt_prev_yr,dev_eldly_prcnt,dev_pct_disabled_lt62_all,dev_pct_lt80_median,dev_median_inc_amnt,dev_scattered_site_ind,dev_pd_status_type_code,dev_total_units,dev_total_dwelling_units,dev_acc_units,dev_total_occupied,dev_regular_vacant,dev_pha_total_units,dev_pct_occupied,dev_number_reported,dev_pct_reported,dev_months_since_report,dev_pct_movein,dev_people_per_unit,dev_people_total,dev_rent_per_month,dev_spending_per_month,dev_hh_income,dev_person_income,dev_pct_lt5k,dev_pct_5k_lt10k,dev_pct_10k_lt15k,dev_pct_15k_lt20k,dev_pct_ge20k,dev_pct_wage_major,dev_pct_welfare_major,dev_pct_other_major,dev_pct_median,dev_pct_lt50_median,dev_pct_lt30_median,dev_pct_2adults,dev_pct_1adult,dev_pct_female_head,dev_pct_female_head_child,dev_pct_disabled_lt62,dev_pct_disabled_ge62,dev_pct_disabled_all,dev_pct_lt24_head,dev_pct_age25_50,dev_pct_age51_61,dev_pct_age62plus,dev_pct_age85plus,dev_pct_minority,dev_pct_black,dev_pct_native_american,dev_pct_asian,dev_pct_hispanic,dev_months_waiting,dev_months_from_movein,dev_pct_utility_allow,dev_ave_util_allow,dev_pct_bed1,dev_pct_bed2,dev_pct_bed3,dev_pct_overhoused,dev_tminority,dev_tpoverty,dev_tpct_ownsfd,pha_phas_designation,pha_ha_low_rent_size_category,pha_section8_units_cnt,pha_ha_section_8_size_category,pha_ha_combined_size_category,pha_ha_fye,pha_ha_program_type,pha_section8_occupied,pha_opfund_amnt,pha_opfund_amnt_prev_yr,pha_capfund_amnt,pha_capfund_amnt_prev_yr,pha_ross_amnt,pha_fss_amnt,pha_chldrn_mbr_cnt,pha_eldly_prcnt,pha_pct_disabled_lt62_all,pha_pct_lt80_median,pha_median_inc_amnt,pha_annl_expns_amnt,pha_annl_expns_amnt_prev_yr,corrected_address,corrected_unit,construction_year,dofa_actual_year
69679,69680,PA002,PA002000902,RW,INAPCP,1,1,1,0,1,100.0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Philadelphia, PA-NJ PMSA",1.0,0.0,PHILADELPHIA,PA,19104.0,1801.0,NaN,1.910418e+10,S,NaN,1.0,90% Overall probable correctness,R,U,N,,2023/03/06 08:50:52+00,NaN,NaN,12799,1297.0,342.0,5706978.0,5305612.0,12.09,14.67,97.309998,12361.0,Y,M,376.0,373.0,373.0,338.0,22.0,12790.0,90.620003,335.0,99.11,12.0,3.88,2.7,893.0,501.0,1407.0,18915.0,7096.0,8.17,24.18,20.59,10.13,36.93,29.25,6.27,55.82,20.04,88.66,74.63,2.39,44.78,90.75,45.07,29.36,53.0,20.94,2.69,47.76,19.7,29.85,2.99,97.01,95.22,0.0,0.0,1.79,0.0,193.0,83.58,187.0,3.88,22.09,74.03,46.57,92.59,43.45,5.36,Advisory,"EXTRA LARGE (5,000+)",23003,"EXTRA LARGE (5,000+)","EXTRA LARGE (5,000+)",3/31,Combined,19008,158748192,129464275,54175579,53271528,0,0,26294.0,14.39,15.25,98.349998,11713.0,379445090.0,337899750.0,609 N 39TH ST,NaN,197

In [34]:
pd.set_option('display.max_rows', 30)
display(public.select_dtypes(include='object') \
    .nunique().sort_values(ascending=False))
pd.set_option('display.max_rows', 10)

corrected_address                 156377
development_code                    6493
std_city                            3444
participant_code                    2754
corrected_unit                      1763
msa_nm                               306
std_st                                54
msgusps                               14
pha_ha_section_8_size_category         6
building_type_code                     6
addr_type                              6
building_status_type_code              6
pha_ha_combined_size_category          6
pha_phas_designation                   6
pha_ha_low_rent_size_category          6
apt_type                               5
lvl2kx                                 5
rad_type                               4
pha_ha_fye                             4
pha_ha_program_type                    2
rad_chap_indr                          2
dev_scattered_site_ind                 2
ur                                     2
zip_class                              2
dev_pd_status_ty

In [35]:
print([x for x in public.columns if 'zip' in x.lower()])

['std_zip5', 'std_zip9', 'zip_class', 'std_zip11']


In [36]:
public['std_zip5'] = public.std_zip5.astype(str)
public['std_zip9'] = public.std_zip9.astype(str)
public['std_zip11'] = public.std_zip11.astype(str)

In [37]:
public['encoded_development_code'] = LabelEncoder().fit_transform(public.development_code)
public['encoded_participant_code'] = LabelEncoder().fit_transform(public.participant_code)
public['encoded_msa_nm'] = LabelEncoder().fit_transform(public.msa_nm)
public['encoded_zip5'] = LabelEncoder().fit_transform(public.std_zip5)
public['encoded_zip9'] = LabelEncoder().fit_transform(public.std_zip9)
public['encoded_zip11'] = LabelEncoder().fit_transform(public.std_zip11)

public = public.drop([
    'development_code',
    'participant_code',
    'msa_nm',
    'std_zip9',
    'std_zip11'
], axis=1)

In [38]:
placeholder = "MISSING_VALUE" 

# List of columns to be one-hot encoded
columns_to_encode = [
    'msgusps',
    'pha_ha_section_8_size_category',
    'building_type_code',
    'addr_type',
    'building_status_type_code',
    'pha_ha_combined_size_category',
    'pha_phas_designation',
    'pha_ha_low_rent_size_category',
    'apt_type',
    'lvl2kx',
    'rad_type',
    'pha_ha_fye',
    'pha_ha_program_type',
    'rad_chap_indr',
    'dev_scattered_site_ind',
    'ur',
    'zip_class',
    'dev_pd_status_type_code',
    'last_updt_dttm'
]

# Fill NaN values with placeholder
for column in columns_to_encode:
    public[column] = public[column].fillna(placeholder)

# One-hot encode the columns
for column in columns_to_encode:
    dummies = pd.get_dummies(public[column], prefix=column)
    public = pd.concat([public, dummies], axis=1)
    public.drop([column], axis=1, inplace=True)

In [39]:
# Encode address info, but don't drop so we can match later
public['encoded_city'] = LabelEncoder().fit_transform(public.std_city)

public['std_st'] = public['std_st'].fillna(placeholder)
dummies = pd.get_dummies(public['std_st'], prefix='std_st')
public = pd.concat([public, dummies], axis=1)

public['encoded_zip5'] = LabelEncoder().fit_transform(public.std_zip5)

In [40]:
public.shape

(191885, 310)

In [41]:
pd.set_option('display.max_rows', 350)
display(public.dtypes.sort_values(ascending=False))
pd.set_option('display.max_rows', 10)

std_st                                                   object
corrected_unit                                           object
corrected_address                                        object
std_zip5                                                 object
std_city                                                 object
dev_pct_5k_lt10k                                        float64
dev_people_total                                        float64
dev_rent_per_month                                      float64
dev_spending_per_month                                  float64
dev_hh_income                                           float64
dev_person_income                                       float64
dev_pct_lt5k                                            float64
dev_median_inc_amnt                                     float64
dev_pct_10k_lt15k                                       float64
dev_pct_movein                                          float64
dev_pct_15k_lt20k                       

Export the cleaned file for future analysis.

In [42]:
public.to_csv('..data/buildings_dev_pha_clean_2.1.csv', index=False)